# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Rest Bedding Co."'

In [12]:
product = "Luxury Air Mattress"
chain.run(product)

'Elevate Air Luxury'

In [13]:
product = "Ohter"
chain.run(product)

'"Other Creations Co."'

## SimpleSequentialChain

In [14]:
from langchain.chains import SimpleSequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Other Creations Co."
Innovative company dedicated to bringing unique, creative products and designs to the market. Passionate about turning ideas into reality.

> Finished chain.


'Innovative company dedicated to bringing unique, creative products and designs to the market. Passionate about turning ideas into reality.'

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [20]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [21]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [22]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [23]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [24]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [25]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and foam quality of the product, suspecting it may be an old batch or counterfeit.',
 'followup_message': "Cher client, nous sommes désolés d'apprendre que vous n'étiez pas satisfait du goût et de la qualité de la mousse de notre produit. Nous vous assurons que nous prenons la qualité de nos produits très au sérieux et nous enquêterons immédiatement sur votre problème. Il est possible qu'il s'agisse d'un lot périmé ou contrefait, et nous ferons tout notre possible pour rectifier la situation. Veuillez nous contacter directement pour que nous puissions résoudre ce problème au plus vi

## Router Chain

In [34]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

englishliterature_template = """ You are a successful English literature expert.\
You have a passion for analyzing themes, characters, and literary devices,\
deep critical thinking, a strong appreciation for storytelling,\
a mastery of historical and cultural contexts, and excellent communication\
skills. You are great at interpreting texts and explaining literary meanings.\
You are so good because you understand how to deconstruct a passage,\
identify its deeper implications, and articulate compelling arguments\
with textual evidence. 

Here is a question:
{input}"""

businessanalytics_template = """ You are a successful business analyst.\
You have a passion for data-driven decision-making,\
strong analytical skills, strategic thinking,\
an understanding of business operations, and excellent communication\
skills. You are great at interpreting business data and deriving insights.\
You are so good because you know how to analyze trends,\
identify key performance indicators, and provide actionable recommendations\
that drive business success. 

Here is a question:
{input}"""


In [35]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "english literature", 
        "description": "Good for answering English literature questions", 
        "prompt_template": englishliterature_template
    },
    {
        "name": "business analytics", 
        "description": "Good for answering business analytics questions", 
        "prompt_template": businessanalytics_template
    }
]


In [36]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [37]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [38]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [39]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [40]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [41]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [42]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [43]:
chain.run("What is reflection of light, and how we can test it?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is reflection of light, and how we can test it?'}
> Finished chain.


'Reflection of light is the phenomenon where light bounces off a surface and changes direction. This can be easily observed in everyday life when looking at a mirror or a shiny surface.\n\nOne way to test reflection of light is to shine a light source, such as a flashlight, at a mirror and observe the angle at which the light reflects off the mirror. The angle of incidence (the angle at which the light hits the mirror) will be equal to the angle of reflection (the angle at which the light bounces off the mirror). This is known as the law of reflection.\n\nAnother way to test reflection of light is to use a laser pointer and shine it at a smooth surface, such as a piece of glass or water. The laser beam will reflect off the surface and create a clear reflection of the beam. This can help demonstrate how light reflects off different surfaces.'

In [44]:
chain.run("what how to calculate divisors of 931")



> Entering new MultiPromptChain chain...
math: {'input': 'what how to calculate divisors of 931'}
> Finished chain.


'To calculate the divisors of 931, we need to find all the numbers that can divide evenly into 931 without leaving a remainder.\n\nOne way to do this is to start by finding the prime factorization of 931. We can do this by dividing 931 by prime numbers starting from 2 until we can no longer divide evenly.\n\n931 ÷ 2 = 465\n465 ÷ 3 = 155\n155 ÷ 5 = 31\n\nSo, the prime factorization of 931 is 2 x 3 x 5 x 31.\n\nTo find the divisors of 931, we can use the prime factorization. We can generate all the divisors by combining the prime factors in different ways. \n\nThe divisors of 931 are:\n1, 2, 3, 5, 6, 10, 15, 31, 62, 93, 155, 186, 310, 465, and 931.\n\nThese are all the numbers that can divide evenly into 931.'

In [45]:
chain.run("What are white blood cells?")



> Entering new MultiPromptChain chain...
None: {'input': 'What are white blood cells?'}
> Finished chain.


'White blood cells, also known as leukocytes, are a type of blood cell that plays a crucial role in the immune system. They help the body fight off infections and diseases by attacking and destroying pathogens such as bacteria, viruses, and other harmful substances. White blood cells are produced in the bone marrow and circulate throughout the body in the blood and lymphatic system. There are several different types of white blood cells, each with its own specific function in the immune response.'

In [46]:
chain.run("Can you speak Spanish / English?")



> Entering new MultiPromptChain chain...
None: {'input': 'Can you speak Spanish / English?'}
> Finished chain.


'Yes, I can speak both Spanish and English. How can I assist you today?'

In [47]:
chain.run("Write me a role description of a Business Administrator")



> Entering new MultiPromptChain chain...
business analytics: {'input': 'Write me a role description of a Business Administrator'}
> Finished chain.


"Role Description: Business Administrator\n\nAs a Business Administrator, you will play a crucial role in ensuring the smooth and efficient operation of the organization. Your responsibilities will include overseeing day-to-day administrative tasks, managing office operations, and providing support to various departments within the company.\n\nKey Responsibilities:\n1. Manage and coordinate administrative functions such as scheduling meetings, handling correspondence, and maintaining office supplies.\n2. Assist in the development and implementation of policies and procedures to improve operational efficiency.\n3. Support department heads in various administrative tasks, including data analysis, report generation, and project coordination.\n4. Collaborate with cross-functional teams to ensure seamless communication and coordination across departments.\n5. Monitor and track key performance indicators to identify areas for improvement and provide actionable recommendations to drive busine

Reminder: Download your notebook to you local computer to save your work.